### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [11]:
# Dependencies and Setup
import pandas as pd

# File to Load (Remember to Change These)
school_data_to_load = "Resources/schools_complete.csv"
student_data_to_load = "Resources/students_complete.csv"

# Read School and Student Data File and store into Pandas DataFrames
school_data = pd.read_csv(school_data_to_load)
student_data = pd.read_csv(student_data_to_load)

# Combine the data into a single dataset.  
school_data_complete = pd.merge(student_data, school_data, how="left", on=["school_name", "school_name"])

## Local Government Area Summary

* Calculate the total number of schools

* Calculate the total number of students

* Calculate the total budget

* Calculate the average maths score 

* Calculate the average reading score

* Calculate the percentage of students with a passing maths score (50 or greater)

* Calculate the percentage of students with a passing reading score (50 or greater)

* Calculate the percentage of students who passed maths **and** reading (% Overall Passing)

* Create a dataframe to hold the above results

* Optional: give the displayed data cleaner formatting

In [12]:
# Calculate number of unique schools in the DataFrame and store as variable
school_count = len(school_data_complete["school_name"].unique())

# Calculate number of students (from unique Student ID) in the DataFrame and store as variable
student_count = len(school_data_complete["Student ID"].unique())

# For debugging - unique student names in student_name
# Counting unique student_names doesn't equate to total students as names can be the same for 2 different individuals on different schools or year level
student_name_unique = len(school_data_complete["student_name"].unique())

# Calculate total budget in the DataFrame and store as variable 
# Determine the unique budget value (as each value is only attributed to a specific school) and calculate the sum of only unique "budget values"
total_budget = school_data_complete["budget"].unique().sum()

# Calculate average maths score and store as variable
maths_average = school_data_complete["maths_score"].mean()

# Calculate average reading score and store as variable
reading_average = school_data_complete["reading_score"].mean()

# Filter using .loc for maths score of >=50 (50 or greater)
# Calculate percentage of students with passing maths score (50 or greater) and store as variable
passing_math_df = school_data_complete.loc[school_data_complete["maths_score"] >= 50]
percent_passing_math = len(passing_math_df["Student ID"].unique()) / student_count * 100

# Filter using .loc for reading score of >=50 (50 or greater)
# Calculate percentage of students with passing reading score (50 or greater) and store as variable
passing_reading_df = school_data_complete.loc[school_data_complete["reading_score"] >= 50]
percent_passing_reading = len(passing_reading_df["Student ID"].unique()) / student_count * 100

# Filter using .loc for both math and reading score of >=50 (50 or greater)
# Calculate percentage of students with both passing maths and reading score (50 or greater) and store as variable
passing_both_df = school_data_complete.loc[(school_data_complete["maths_score"] >= 50) & 
                                           (school_data_complete["reading_score"] >= 50)]
percent_passing_both = len(passing_both_df["Student ID"].unique()) / student_count * 100

# Create dataframe to hold these results
area_summary = pd.DataFrame({"Total Schools": [school_count],
                             "Total Students": [student_count],
                             "Total Budget": [total_budget],
                             "Average Maths Score": [maths_average],
                             "Average Reading Score": [reading_average],
                             "% Passing Maths": [percent_passing_math],
                             "% Passing Reading": [percent_passing_reading],
                             "% Overall Passing": [percent_passing_both]})

# Format dataframe 
area_summary["Total Budget"] = area_summary["Total Budget"].map("${:,.2f}".format)

# Format style by text-align to left
area_summary = area_summary.style.set_properties(**{'text-align': 'left'})

area_summary

,Total Schools,Total Students,Total Budget,Average Maths Score,Average Reading Score,% Passing Maths,% Passing Reading,% Overall Passing
0,15,39170,"$24,649,428.00",70.338192,69.980138,86.078632,84.426857,72.808272


## School Summary

* Create an overview table that summarises key metrics about each school, including:
  * School Name
  * School Type
  * Total Students
  * Total School Budget
  * Per Student Budget
  * Average Maths Score
  * Average Reading Score
  * % Passing Maths
  * % Passing Reading
  * % Overall Passing (The percentage of students that passed maths **and** reading.)
  
* Create a dataframe to hold the above results

In [13]:
# Create dataframe grouped by school name and type and store as variable
school_avg_df = school_data_complete.groupby(["school_name", "type"]).mean(numeric_only=True)

# Create column in school_avg_df dataframe for "per student budget" with formula of budget / size
school_avg_df["Per Student Budget"] = \
        school_avg_df["budget"] / \
        school_avg_df["size"]

# Take column for size and store as variable
summary_size = school_avg_df["size"]

# Take column for budget and store as variable
summary_budget = school_avg_df["budget"]

# Take column for reading_score and store as variable
summary_reading_avg = school_avg_df["reading_score"]

# Take column for maths_score and store as variable
summary_math_avg = school_avg_df["maths_score"]

# Take column for per student budget and store as variable
summary_perbudget = school_avg_df["Per Student Budget"]

# From passing_math_df, create dataframe grouped by school name and type, 
# with passing maths score (50 or greater), and get the number of students and store as variable
math_avg_df = passing_math_df.groupby(["school_name", "type"]).count()

# Take column for passing maths score (50 or greater) and store as variable
math_avg_df["maths_score"]

# Take column for passing maths score from math_avg_df
# Divide that amount from total size from school_avg_df to calculate % Passing maths and store as variable
summary_math_percent = math_avg_df.loc[:,("maths_score")] / school_avg_df.loc[:, ("size")] *100

# From passing_reading_df, create dataframe grouped by school name and type, 
# with passing reading score (50 or greater), and get the number of students and store as variable
reading_avg_df = passing_reading_df.groupby(["school_name", "type"]).count()

# Take column for passing reading score from reading_avg_df
# Divide that amount from total size from school_avg_df to calculate % Passing reading and store as variable
summary_reading_percent = reading_avg_df.loc[:, ("reading_score")] / school_avg_df.loc[:,("size")] *100

# From passing_both_df, create dataframe grouped by school name and type, 
# with passing both maths and reading score (50 or greater), and get the number of students and store as variable
both_avg_df = passing_both_df.groupby(["school_name", "type"]).count()

# Take column for passing maths or reading score (both are the same numbers) from both_avg_df
# Divide that amount from total size from school_avg_df to calculate % Overall Passing and store as variable
summary_both_percent = both_avg_df.loc[:, ("maths_score")] / school_avg_df.loc[:, ("size")] *100

# Remove unrelated columns from school_avg_df to only include the required columns and store as variable
summary_school_df = school_avg_df[["size", 
                                   "budget",
                                   "Per Student Budget", 
                                   "maths_score", 
                                   "reading_score"]]

# Add the required columns to the summary_school_df
summary_school_df["% Passing Maths"] = summary_math_percent
summary_school_df["% Passing Reading"] = summary_reading_percent
summary_school_df["% Overall Passing"] = summary_both_percent

# Reset index for formatting
summary_school_reset_df = summary_school_df.reset_index()

# Renaming columns for consistency
per_school_summary = summary_school_reset_df.rename(columns={
                            "size":"Total Students",
                            "school_name":"",
                            "budget":"Total School Budget",
                            "maths_score":"Average Maths Score",
                            "reading_score":"Average Reading Score"})

# Format cell values 
per_school_summary["Total Students"] = per_school_summary["Total Students"].map("{:.0f}".format)
per_school_summary["Total School Budget"] = per_school_summary["Total School Budget"].map("${:,.2f}".format)
per_school_summary["Per Student Budget"] = per_school_summary["Per Student Budget"].map("${:,.2f}".format)

per_school_summary

C:\Users\Caroline\AppData\Local\Temp\ipykernel_1104\3450951741.py:59: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  summary_school_df["% Passing Maths"] = summary_math_percent
C:\Users\Caroline\AppData\Local\Temp\ipykernel_1104\3450951741.py:60: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  summary_school_df["% Passing Reading"] = summary_reading_percent


,,type,Total Students,Total School Budget,Per Student Budget,Average Maths Score,Average Reading Score,% Passing Maths,% Passing Reading,% Overall Passing
0,Bailey High School,Government,4976,"$3,124,928.00",$628.00,72.352894,71.008842,91.639871,87.379421,80.084405
1,Cabrera High School,Independent,1858,"$1,081,356.00",$582.00,71.657158,71.359526,90.850377,89.074273,80.785791
2,Figueroa High School,Government,2949,"$1,884,411.00",$639.00,68.698542,69.077993,81.654798,82.807731,67.650051
3,Ford High School,Government,2739,"$1,763,916.00",$644.00,69.091274,69.572472,82.438846,82.219788,67.469880
4,Griffin High School,Independent,1468,"$917,500.00",$625.00,71.788147,71.245232,91.212534,88.487738,81.335150
5,Hernandez High School,Government,4635,"$3,022,020.00",$652.00,68.874865,69.186408,80.949299,81.877023,66.364617
6,Holden High School,Independent,427,"$248,087.00",$581.00,72.583138,71.660422,89.929742,88.524590,78.922717
7,Huang High School,Government,2917,"$1,910,635.00",$655.00,68.935207,68.910525,81.693521,81.453548,66.712376
8,Johnson High School,Government,4761,"$3,094,650.00",$650.00,68.843100,69.039277,82.062592,81.978576,67.191766
9,Pena High School,Independent,962,"$585,858.00",$609.00,72.088358,71.613306,91.683992,86.590437,79.209979


## Top Performing Schools (By % Overall Passing)

* Sort and display the top five performing schools by % overall passing.

In [14]:
# Sort % Overall Passing values, highest to lowest
top_schools = per_school_summary.sort_values(by=["% Overall Passing"], ascending=False).reset_index(drop=True, inplace=False)
top_schools.head(5)

,,type,Total Students,Total School Budget,Per Student Budget,Average Maths Score,Average Reading Score,% Passing Maths,% Passing Reading,% Overall Passing
0,Griffin High School,Independent,1468,"$917,500.00",$625.00,71.788147,71.245232,91.212534,88.487738,81.335150
1,Cabrera High School,Independent,1858,"$1,081,356.00",$582.00,71.657158,71.359526,90.850377,89.074273,80.785791
2,Bailey High School,Government,4976,"$3,124,928.00",$628.00,72.352894,71.008842,91.639871,87.379421,80.084405
3,Wright High School,Independent,1800,"$1,049,400.00",$583.00,72.047222,70.969444,91.777778,86.666667,79.722222
4,Rodriguez High School,Government,3999,"$2,547,363.00",$637.00,72.047762,70.935984,90.797699,87.396849,79.419855


## Bottom Performing Schools (By % Overall Passing)

* Sort and display the five worst-performing schools by % overall passing.

In [15]:
# Sort % Overall Passing values, lowest to highest
bottom_schools = per_school_summary.sort_values(by=["% Overall Passing"], ascending=True).reset_index(drop=True, inplace=False)
bottom_schools.head(5)

,,type,Total Students,Total School Budget,Per Student Budget,Average Maths Score,Average Reading Score,% Passing Maths,% Passing Reading,% Overall Passing
0,Hernandez High School,Government,4635,"$3,022,020.00",$652.00,68.874865,69.186408,80.949299,81.877023,66.364617
1,Huang High School,Government,2917,"$1,910,635.00",$655.00,68.935207,68.910525,81.693521,81.453548,66.712376
2,Johnson High School,Government,4761,"$3,094,650.00",$650.00,68.843100,69.039277,82.062592,81.978576,67.191766
3,Wilson High School,Independent,2283,"$1,319,574.00",$578.00,69.170828,68.876916,82.785808,81.296540,67.455103
4,Ford High School,Government,2739,"$1,763,916.00",$644.00,69.091274,69.572472,82.438846,82.219788,67.469880


## Maths Scores by Year

* Create a table that lists the average maths score for students of each year level (9, 10, 11, 12) at each school.

  * Create a pandas series for each year. Hint: use a conditional statement.
  
  * Group each series by school
  
  * Combine the series into a dataframe
  
  * Optional: give the displayed data cleaner formatting

In [16]:
# Create dataframe that get all values for year 9, grouped by school name
# Calculate year 9 average maths_score and store as variable
y9_math_df = school_data_complete[school_data_complete["year"] == 9].groupby("school_name")["maths_score"].mean().reset_index()
y9_math = y9_math_df["maths_score"]

# Create dataframe that get all values for year 10, grouped by school name
# Calculate year 10 average maths_score and store as variable
y10_math_df = school_data_complete[school_data_complete["year"] == 10].groupby("school_name")["maths_score"].mean().reset_index()
y10_math = y10_math_df["maths_score"]

# Create dataframe that get all values for year 11, grouped by school name
# Calculate year 11 average maths_score and store as variable
y11_math_df = school_data_complete[school_data_complete["year"] == 11].groupby("school_name")["maths_score"].mean().reset_index()
y11_math = y11_math_df["maths_score"]

# Create dataframe that get all values for year 12, grouped by school name
# Calculate year 12 average maths_score and store as variable
y12_math_df = school_data_complete[school_data_complete["year"] == 12].groupby("school_name")["maths_score"].mean().reset_index()
y12_math = y12_math_df["maths_score"]

# Rename column name math_scores to Year 9, remove school name column name and rename y9_math_df to math_by_year
maths_scores_by_year = y9_math_df.rename(columns={"maths_score":"Year 9",
                                          "school_name":""})

# Add years 10, 11 & 12 columns to the math_by_year
maths_scores_by_year["Year 10"] = y10_math
maths_scores_by_year["Year 11"] = y11_math
maths_scores_by_year["Year 12"] = y12_math

maths_scores_by_year

,,Year 9,Year 10,Year 11,Year 12
0,Bailey High School,72.493827,71.897498,72.374900,72.675097
1,Cabrera High School,72.321970,72.437768,71.008299,70.604712
2,Figueroa High School,68.477804,68.331586,68.811001,69.325282
3,Ford High School,69.021609,69.387006,69.248862,68.617811
4,Griffin High School,72.789731,71.093596,71.692521,71.469178
5,Hernandez High School,68.586831,68.867156,69.154412,68.985075
6,Holden High School,70.543307,75.105263,71.640777,73.409639
7,Huang High School,69.081754,68.533246,69.431345,68.639316
8,Johnson High School,69.469286,67.990220,68.637730,69.287393
9,Pena High School,71.996364,72.396000,72.523438,71.187845


## Reading Score by Year

* Perform the same operations as above for reading scores

In [17]:
# Create dataframe that get all values for year 9, grouped by school name
# Calculate year 9 average reading_score and store as variable
y9_reading_df = school_data_complete[school_data_complete["year"] == 9].groupby("school_name")["reading_score"].mean().reset_index()
y9_reading = y9_reading_df["reading_score"]

# Create dataframe that get all values for year 10, grouped by school name
# Calculate year 10 average reading_score and store as variable
y10_reading_df = school_data_complete[school_data_complete["year"] == 10].groupby("school_name")["reading_score"].mean().reset_index()
y10_reading = y10_reading_df["reading_score"]

# Create dataframe that get all values for year 11, grouped by school name
# Calculate year 11 average reading_score and store as variable
y11_reading_df = school_data_complete[school_data_complete["year"] == 11].groupby("school_name")["reading_score"].mean().reset_index()
y11_reading = y11_reading_df["reading_score"]

# Create dataframe that get all values for year 12, grouped by school name
# Calculate year 12 average reading_score and store as variable
y12_reading_df = school_data_complete[school_data_complete["year"] == 12].groupby("school_name")["reading_score"].mean().reset_index()
y12_reading = y12_reading_df["reading_score"]

# Rename column name math_scores to Year 9, remove school name column name and rename y9_math_df to math_by_year
reading_scores_by_year = y9_reading_df.rename(columns={"reading_score":"Year 9",
                                                "school_name":""})

# Add years 10, 11 & 12 columns to the math_by_year
reading_scores_by_year ["Year 10"] = y10_reading
reading_scores_by_year ["Year 11"] = y11_reading
reading_scores_by_year ["Year 12"] = y12_reading

reading_scores_by_year 

,,Year 9,Year 10,Year 11,Year 12
0,Bailey High School,70.901920,70.848265,70.317346,72.195525
1,Cabrera High School,71.172348,71.328326,71.201245,71.856021
2,Figueroa High School,70.261682,67.677588,69.152327,69.082126
3,Ford High School,69.615846,68.988701,70.735964,68.849722
4,Griffin High School,72.026895,70.746305,72.385042,69.434932
5,Hernandez High School,68.477569,70.621842,68.418199,69.244136
6,Holden High School,71.598425,71.096491,73.310680,70.481928
7,Huang High School,68.670616,69.516297,68.740638,68.671795
8,Johnson High School,68.719286,69.295029,69.969115,67.992521
9,Pena High School,70.949091,72.324000,71.703125,71.513812


## Scores by School Spending

* Create a table that breaks down school performances based on average Spending Ranges (Per Student). Use 4 reasonable bins to group school spending. Include in the table each of the following:
  * Average Maths Score
  * Average Reading Score
  * % Passing Maths
  * % Passing Reading
  * Overall Passing Rate (Average of the above two)

In [18]:
# Create bins in which to place values based per student budget
bins = [0, 585, 630, 645, 680]

# Create labels for these bins
group_labels = ["<$585", "$585-630", "$630-645", "$645-680"]

# Use pd.cut to store "per Student Budget" into 4 bins, with group labels.
summary_school_reset_df["Spending Range"] = pd.cut(summary_school_reset_df["Per Student Budget"], bins, labels=group_labels)

# Create dataframe to group "Spending Range" and calculate the average and store in variable
spending_summary = summary_school_reset_df.groupby(["Spending Range"]).mean(numeric_only=True).reset_index()

# Remove unrelated columns
spending_summary = spending_summary[["Spending Range",
                                     "maths_score", 
                                     "reading_score",
                                     "% Passing Maths", 
                                     "% Passing Reading",
                                     "% Overall Passing"]]

# Rename columns for consistency
spending_summary = spending_summary.rename(columns={"maths_score":"Average Maths Score",
                                                    "reading_score":"Average Reading Score"})

# Format numbers to 2 decimal places
spending_summary["Average Maths Score"] = spending_summary["Average Maths Score"].map("{:.2f}".format)
spending_summary["Average Reading Score"] = spending_summary["Average Reading Score"].map("{:.2f}".format)
spending_summary["% Passing Maths"] = spending_summary["% Passing Maths"].map("{:.2f}".format)
spending_summary["% Passing Reading"] = spending_summary["% Passing Reading"].map("{:.2f}".format)
spending_summary["% Overall Passing"] = spending_summary["% Overall Passing"].map("{:.2f}".format)

spending_summary

,Spending Range,Average Maths Score,Average Reading Score,% Passing Maths,% Passing Reading,% Overall Passing
0,<$585,71.36,70.72,88.84,86.39,76.72
1,$585-630,72.07,71.03,91.52,87.29,79.88
2,$630-645,69.85,69.84,84.69,83.76,71.00
3,$645-680,68.88,69.05,81.57,81.77,66.76


## Scores by School Size

* Perform the same operations as above, based on school size.

In [19]:
# Create bins in which to place values based size
bins = [0, 1000, 2000, 5000]

# Create labels for these bins
group_labels = ["Small (<1000)", "Medium (1000-2000)", "Large (2000-5000)"]

# Use pd.cut to store "sizes" into 3 bins, with group labels.
summary_school_reset_df["School Size"] = pd.cut(summary_school_reset_df["size"], bins, labels=group_labels)

# Create dataframe to group "School Size" and calculate the average and store in variable
size_summary = summary_school_reset_df.groupby(["School Size"]).mean(numeric_only=True).reset_index()

# Remove unrelated columns
size_summary = size_summary[["School Size",
                             "maths_score", 
                             "reading_score",
                             "% Passing Maths", 
                             "% Passing Reading",
                             "% Overall Passing"]]

# Rename columns for consistency
size_summary = size_summary.rename(columns={"maths_score":"Average Maths Score",
                                            "reading_score":"Average Reading Score"})

size_summary

,School Size,Average Maths Score,Average Reading Score,% Passing Maths,% Passing Reading,% Overall Passing
0,Small (<1000),72.335748,71.636864,90.806867,87.557513,79.066348
1,Medium (1000-2000),71.421650,70.720164,89.846560,86.714149,78.039785
2,Large (2000-5000),69.751809,69.576052,84.252804,83.301185,70.293507


## Scores by School Type

* Perform the same operations as above, based on school type

In [20]:
# Create dataframe to group "Type" and calculate the average and store in variable
type_summary = summary_school_reset_df.groupby(["type"]).mean(numeric_only=True).reset_index()

# Remove unrelated columns
type_summary = type_summary[["type",
                             "maths_score", 
                             "reading_score",
                             "% Passing Maths", 
                             "% Passing Reading",
                             "% Overall Passing"]]

# Rename columns for consistency
type_summary = type_summary.rename(columns={"maths_score":"Average Maths Score",
                                            "reading_score":"Average Reading Score",
                                            "type":"School Type"})

type_summary

,School Type,Average Maths Score,Average Reading Score,% Passing Maths,% Passing Reading,% Overall Passing
0,Government,69.834806,69.675929,84.462375,83.587562,70.698993
1,Independent,71.368822,70.718933,89.204043,86.247789,76.973340
